In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [5]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [6]:
special_events = pd.read_csv('Datasources/other general/special_events_list_datasd.csv')

In [7]:
schema = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [8]:
schema_two = StructType([StructField("event_id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [9]:
spql_listings_info = sqlContext.createDataFrame(listings[['id', 'longitude', 'latitude']], schema)

In [10]:
spql_special_events_info = sqlContext.createDataFrame(special_events[['event_id', 'longitude', 'latitude']].dropna(), schema_two)

In [11]:
spql_listings_info_rdd = spql_listings_info.rdd.map(lambda row:(row.id, row.longitude,row.latitude))

In [12]:
spql_special_events_info_rdd = spql_special_events_info.rdd.map(lambda row:(row.event_id, row.longitude,row.latitude))

In [13]:
spql_listings_and_events_info_rdd = spql_listings_info_rdd.cartesian(spql_special_events_info_rdd)

In [14]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_rdd.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1][0], x[1][1], x[1][2]))

In [15]:
def distance_calculation(x):
        p_one = float(x[2])
        q_one = float(x[1])
        p_two = float(x[5])
        q_two = float(x[4])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371.00*float(c)
        return tuple(list(x) + [d])

In [16]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_rdd.map(distance_calculation)

In [17]:
schema_three = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)] + [StructField("event_id", IntegerType(), True),
                    StructField("event_longitude", DoubleType(), True),
                    StructField("event_latitude", DoubleType(), True), StructField("distance", DoubleType(), True)])

In [18]:
spql_listings_and_events_info_df = sqlContext.createDataFrame(spql_listings_and_events_info_rdd, schema_three)

In [19]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_df.rdd.map(lambda row: (row.id, row.longitude, row.latitude, row.event_id, row.event_longitude, row.event_latitude, row.distance))

In [20]:
closest_event_distance = spql_listings_and_events_info_rdd.map(lambda x: (x[0], x[-1])).reduceByKey(lambda x, y : min(x, y))

In [22]:
close_events_count_one = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 1).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_three = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 3).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_five = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 5).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_ten = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 10).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [23]:
event_distance  = closest_event_distance.map(lambda x: x[1])

In [25]:
close_events_count = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 16).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [ ]:
close_events_count_three_val = close_events_count_three.map(lambda x: x[1])
close_events_count_five_val = close_events_count_five.map(lambda x: x[1])
close_events_count_ten_val = close_events_count_ten.map(lambda x: x[1])
close_events_count_three_val = close_events_count_three.map(lambda x: x[1])

In [26]:
close_events_count_all = close_events_count_one.zip(close_events_count_three.map(lambda x: x[1]))
close_events_count_all = close_events_count_all.zip(close_events_count_five.map(lambda x: x[1]))
close_events_count_all = close_events_count_all.zip(close_events_count_ten.map(lambda x: x[1]))
close_events_count_all = close_events_count_all.zip(close_events_count.map(lambda x: x[1]))

spql_listings_with_events_info_rdd = close_events_count_all.zip(event_distance).map(lambda x: (x[0][0], x[0][1], x[1], x[2], x[3], x[4], x[5]))

In [27]:
spql_listings_with_events_info_rdd.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 55, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\opt\spark\python\lib\pyspark.zip\pyspark\worker.py", line 174, in main
  File "C:\opt\spark\python\lib\pyspark.zip\pyspark\worker.py", line 169, in process
  File "C:\opt\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 220, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "C:\opt\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 138, in dump_stream
    for obj in iterator:
  File "C:\opt\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 209, in _batched
    for item in iterator:
  File "C:\opt\spark\python\lib\pyspark.zip\pyspark\serializers.py", lin

In [ ]:
spql_listings_with_events_info_rdd_row = spql_listings_with_events_info_rdd.map(lambda x: Row(listing_id = x[0], events_count_one = x[1], events_count_three = x[2], events_count_five = x[3], events_count_ten = x[4], events_count = x[5], closest_distance = x[6]))